<a href="https://colab.research.google.com/github/Mario-RJunior/olist-e-commerce/blob/master/PCA_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Análise dos Componentes Principais (PCA)

Neste notebook iremos explorar as compras realizadas do ponto de vista dos tipos de produtos, ou seja, levando em consideração o departamento (categoria) a que eles se encontram. Para isso abordaremos uma técnica denominada Análise dos Componentes Principais (do inglês PCA - Principal Components Analysis) e nosso objetivo é tirar insigths acerca das vendas de cada departamento.

In [1]:
# Importando as bibliotecas
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
! git clone https://github.com/Mario-RJunior/olist-e-commerce

Cloning into 'olist-e-commerce'...
remote: Enumerating objects: 127, done.
remote: Counting objects: 100% (127/127), done.
remote: Compressing objects: 100% (127/127), done.
remote: Total 308 (delta 80), reused 0 (delta 0), pack-reused 181
Receiving objects: 100% (308/308), 26.44 MiB | 23.73 MiB/s, done.
Resolving deltas: 100% (180/180), done.


- Carregando os dataframes

In [8]:
# Criando os dataframes a serem usados
customer = pd.read_csv('olist-e-commerce/datasets/olist_customers_dataset.csv')
orders = pd.read_csv('olist-e-commerce/datasets/olist_orders_dataset.csv', 
                     usecols=['order_id', 'customer_id'])
orders_items = pd.read_csv('olist-e-commerce/datasets/olist_order_items_dataset.csv', 
                           usecols=['order_id', 'order_item_id', 'product_id', 'price', 'freight_value'])
products = pd.read_csv('olist-e-commerce/datasets/olist_products_dataset.csv', 
                   usecols=['product_id', 'product_category_name'])

In [4]:
customer.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP


In [5]:
orders.head()

,order_id,customer_id
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c


In [6]:
orders_items.head()

,order_id,order_item_id,product_id,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,199.90,18.14


In [9]:
products.head()

,product_id,product_category_name
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria
1,3aa071139cb16b67ca9e5dea641aaa2f,artes
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer
3,cef67bcfe19066a932b7673e239eb23d,bebes
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas


In [13]:
print(f'Shape dos dataframes: \ncustomer: {customer.shape}\n'
      f'orders: {orders.shape} \norders_items: {orders_items.shape}\n'
      f'products: {products.shape}')

Shape dos dataframes: 
customer: (99441, 5)
orders: (99441, 2) 
orders_items: (112650, 5)
products: (32951, 2)


- Fazendo um merge dos dataframes acima

Podemos agora gerar um dataframe apenas que consiste em uma junção dos quatro carregados anteriormente.

In [26]:
# Merge dos dataframes
df = pd.merge(left=prod,
             right=orders_items,
             on='product_id')
df = pd.merge(left=df,
             right=orders,
             on='order_id')
df = pd.merge(left=df,
             right=customer,
             on='customer_id')
df.head()

,product_id,product_category_name,order_id,order_item_id,price,freight_value,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,e17e4f88e31525f7deef66779844ddce,1,10.91,7.39,f8a3e963a310aa58b60a5b1fed5bceb5,b1a1199364a4a7fe27c4486ab63f550d,13848,mogi-guacu,SP
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,5236307716393b7114b53ee991f36956,1,248.00,17.99,03fc97548af8f58fefc768d12b546c9c,4b86049cb99e4aa774031daa9cd18f18,20551,rio de janeiro,RJ
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,01f66e58769f84129811d43eefd187fb,1,79.80,7.82,e41819d1c95c12c9ce495b630eab8aee,f63805d9c7edb84d92413af34b86a39c,5821,sao paulo,SP
3,cef67bcfe19066a932b7673e239eb23d,bebes,143d00a4f2dde4e0364ee1821577adb3,1,112.30,9.54,322162b5ca010c2b059cb5224dd818b1,619e926d09b26efbd5180368b1ddc874,2018,sao paulo,SP
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,86cafb8794cb99a9b1b77fc8e48fbbbb,1,37.90,8.29,c11c31965ff02cc1d7132df8edfcbc22,ad353b4fb0e294adc4eda48af73e68a6,5835,sao paulo,SP


In [27]:
df.shape

(112650, 11)

- Verificando valores missing

In [28]:
df.isnull().sum()

product_id                     0
product_category_name       1603
order_id                       0
order_item_id                  0
price                          0
freight_value                  0
customer_id                    0
customer_unique_id             0
customer_zip_code_prefix       0
customer_city                  0
customer_state                 0
dtype: int64

Note que há 1603 valores faltantes no dataframe, todos localizados na coluna product_category_name. Então, inicialmente iremos testar nosso modelo de PCA deletando estes valores. Note que neste caso temos um dataframe com 112650 registros e portanto estaremos retirando uma quantidade correspondente a menos de 2% dele, não sendo então uma quantidade tão significativa a ponto de prejudicar o modelo.

In [31]:
# Cálculo da porcentagem de dados retirados
dados_retirados = 1603
quantidade_total = 112650
porcentagem = (dados_retirados / quantidade_total) * 100
print(f'Dados retirados: {round(porcentagem, 2)}%.')

Dados retirados: 1.42%.
